In [ ]:
%run base.ipynb

In [ ]:
experiment = 'all-mpnet-baseline'
experiment = 'all-mpnet-mulneg'
# experiment = 'all-mpnet-mulneg-beginning'


results_path = get_results_path(experiment)

In [ ]:
from kialo_util import read_maps
maps = read_maps(results_path / 'all/all_maps_embedding.pkl')
samples = read_maps(results_path / 'annotated_samples_embeddings.pkl')
# maps = read_maps(Path('/home/ij/Downloads/Code/deliberatorium/results/results/annotated_samples_embeddings.pkl'))

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

def tsne_plot(embeddings, labels, weights, output=None, extra_embeddings=None, extra_weights=None,):    
#     tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=500, random_state=42)
#     tsne_model = TSNE(perplexity=250*(2 if extra_embeddings else 1), n_components=2, init='pca', n_iter=1000, random_state=42)
    tsne_model = TSNE(perplexity=500, n_components=2, init='pca', n_iter=1000, random_state=42)
    emb2d = tsne_model.fit_transform(embeddings if not extra_embeddings else embeddings+extra_embeddings)
        
    fig = plt.figure(figsize=(10, 10)) 
    x, y = emb2d[:,0],emb2d[:,1]
    if not extra_embeddings:
        g = plt.scatter(x, y, c=weights, cmap='Blues', vmin=0, alpha=0.5)
    else:
        plt.scatter(x[len(embeddings):], y[len(embeddings):], c=extra_weights, cmap='Oranges', vmin=0, alpha=0.5)
        plt.scatter(x[:len(embeddings)], y[:len(embeddings)], c=weights, cmap='Blues', vmin=0, alpha=0.5)
#         plt.colorbar()
#         plt.show()
#         plt.figure(figsize=(10, 10)) 
    
    if len(labels) < 100:
        for i in range(len(x)):
            plt.annotate(labels[i][:100],
                         xy=(x[i], y[i]),
                         xytext=(5, 2),
                         textcoords='offset points',
                         ha='right',
                         va='bottom')
            
#     plt.colorbar()

    fig.savefig(f'output/{experiment}-{output if output else ""}.pdf', format='pdf')
    
    plt.show()

In [ ]:
sample = list(samples.values())[0]
tsne_plot(np.vstack([sample['candidates_embedding'], sample['embedding']]), 
          [x['text'] for x in list(sample['candidates'].values())]+[sample['text']], ([1]*len(sample['candidates'])+[5]), output='samples')
print(sample['text'])
print(sample['candidates'][sample['parent ID']]['text'])

In [ ]:
# np.array([x.embedding for x in maps[0].all_children]).shape
# tsne_plot([x.embedding for x in maps[0].all_children], [x.get_level() for x in maps[0].all_children])
if not 'beginning' in experiment:
    tsne_plot([x.embedding for x in maps[0].all_children], [x.name for x in maps[0].all_children], [10-x.get_level() for x in maps[0].all_children], output='map')
else:
    tsne_plot([x.embedding for x in maps[0].all_children], 
              [x.name for x in maps[0].all_children], 
              [10-x.get_level() for x in maps[0].all_children],
              extra_embeddings=[x.extra_embeddings['parent'] for x in maps[0].all_children],
              extra_weights=[10-x.get_level() for x in maps[0].all_children],
              output='map'
         )